In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from looptools.vectfit3 import opts, vectfit
from looptools.component import Component
from control import ss2tf, tf
from io import StringIO
from scipy.constants import pi
from scipy.signal import cont2discrete
from scipy.signal import ss2tf as scipy_ss2tf

In [ ]:
N=101                                              # Number of frequency samples
s=2j*pi*np.logspace(0,4,N,dtype=np.complex128)     # Samples in the frequency domain
f=np.zeros(N,dtype=np.complex128)                  # Function samples in the frequency domain
weights=np.ones(N,dtype=np.float64)                # Common and unitary weiths for all frequency samples

# Generating test function values
n=0       
for sn in s:
    f[n]=2/(sn+5)+(30+40j)/(sn-(-100+500j))+(30-40j)/(sn-(-100-500j))+0.5
    n+=1

print("\nFrequency domain samples of f(s) = \n",f)
print("f(s) shape = ",f.shape, "\ndata type in f(s) = ",type(f),type(f[0]))

n=3                                                # Order of aproximation
poles=-2*pi*np.logspace(0,4,n,dtype=np.complex128) # Initial searching poles

print("\nInitial searching poles:\n",poles)

# vector fitting configuration
opts["asymp"]=3        # Modified to include D and E in fitting
opts["phaseplot"]=True # Modified to include the phase angle graph

# Remaining options by default

print("\n * Applying vector fitting...")
(SER,poles,rmserr,fit)=vectfit(f,s,poles,weights,opts)
print(" v/ Fitting process completed. Aproximation error achieved = ",rmserr)
print("\nFinal poles computed:\n",poles)

In [ ]:
input_tf = """
9.99991201e-01	-6.6016e+00	0
1.43695081e+00	-6.6016e+00	0
2.06484580e+00	-6.6016e+00	-0.759999999999991
2.96710796e+00	-6.6016e+00	-0.180000000000007
4.26362570e+00	-6.6016e+00	-0.0500000000000114
6.12667432e+00	-6.6592e+00	-0.389999999999986
8.80380709e+00	-6.6972e+00	-0.319999999999993
1.26507491e+01	-6.7939e+00	-0.530000000000001
1.81786641e+01	-6.8407e+00	-1.08000000000001
2.61220759e+01	-6.9272e+00	-1.86000000000001
3.75364685e+01	-6.9918e+00	-2.94
5.39385334e+01	-7.1844e+00	-3.38
7.75077013e+01	-7.2851e+00	-5.22
1.11375734e+02	-7.5547e+00	-7.06999999999999
1.60042860e+02	-7.8236e+00	-9.19
2.29975741e+02	-8.3011e+00	-11.98
3.30466735e+02	-8.8473e+00	-14.28
4.74868621e+02	-9.4829e+00	-15.97
6.82368855e+02	-1.0246e+01	-17.66
9.80539107e+02	-1.1041e+01	-18.37
1.40899886e+03	-1.1757e+01	-18.86
2.02467987e+03	-1.2375e+01	-19.47
2.90939099e+03	-1.2962e+01	-20.36
4.18068852e+03	-1.3532e+01	-23.28
6.00749662e+03	-1.4034e+01	-27.48
8.63255310e+03	-1.4801e+01	-33.44
1.24046633e+04	-1.5615e+01	-41.87
1.78250479e+04	-1.6764e+01	-57.42
2.56139424e+04	-1.8021e+01	-73.19
3.68062991e+04	-1.9335e+01	-94.111
5.28893068e+04	-2.0953e+01	-124.513
7.60000010e+04	-2.3507e+01	-159.748
"""
df = pd.read_csv(StringIO(input_tf.strip()), sep='\t', header=None, names=['freq', 'gain', 'phase'])

# Frequency domain conversion
frfr = np.array(df['freq'], np.float64)
omega = 2 * pi * frfr
gain = (1/1e-9) * 10 ** (df['gain'].values / 20)
phase_rad = np.deg2rad(df['phase'].values)
F = gain * np.exp(1j * phase_rad)
F = F / np.max(np.abs(F))
s = 1j * omega

# Linearly decaying weights or uniform
N = len(F)
# weights = np.linspace(1.0, 0.1, N)
weights = np.ones(N)

# Initial poles (complex conjugate pairs, log-distributed)
n_poles = 4
Bet = np.logspace(np.log10(omega[0]), np.log10(omega[-1]), n_poles // 2)
poles = np.zeros(n_poles, dtype=np.complex128)

# for k in range(n_poles // 2):
#     alf = -Bet[k] / 100
#     poles[2 * k] = alf - 1j * Bet[k]
#     poles[2 * k + 1] = alf + 1j * Bet[k]

# More damping for stability
for k in range(n_poles // 2):
    omega_k = Bet[k]
    sigma = -2 * pi * omega_k * 0.1  # 10% damping
    poles[2*k]   = sigma + 1j * omega_k
    poles[2*k+1] = sigma - 1j * omega_k
    
# Push left in s-plane (more damping)
pole_shift = 2 * pi * 1e5  # e.g., 100 kHz
poles += -pole_shift

# Configure and run vectfit
opts["asymp"] = 2
opts["phaseplot"] = True
opts["spy2"] = False
opts["skip_res"] = True
opts["cmplx_ss"] = False

Niter = 6 # Iterative approach
for itr in range(Niter):
    if itr == Niter - 1:
        opts["spy2"] = True
        opts["skip_res"] = False
    SER, poles, rmserr, fit = vectfit(F, s, poles, weights, opts)

In [ ]:
sps = 80e6

# Extract matrices from SER
A = np.atleast_2d(SER["A"])
B = np.atleast_2d(SER["B"])
C = np.atleast_2d(SER["C"])
D = np.atleast_2d(SER["D"])

# Ensure proper shapes
if B.ndim == 1:
    B = B[:, np.newaxis]
if C.shape[0] != 1:
    C = C.reshape(1, -1)
if D.shape != (1, 1):
    D = D.reshape(1, 1)

# Generate TransferFunction from state-space
H_tf = ss2tf(A, B, C, D)

# Add proportional term if present
E = float(SER.get("E", 0.0))
if E != 0.0:
    H_tf = H_tf + tf([E, 0], [1])

print(H_tf)

# Discretize using Tustin (bilinear)
Ad, Bd, Cd, Dd, dt = cont2discrete((A, B, C, D), dt=1/sps, method='bilinear')
num_d, den_d = scipy_ss2tf(Ad, Bd, Cd, Dd)

com = Component(name="VF_fitted", sps=sps, tf=(num_d.flatten(), den_d.flatten()))

fig, ax = com.bode_plot(frfr, label='Fitted and transformed')
ax[0].semilogx(frfr, np.abs(F), ls='--', label='Measured')
ax[1].semilogx(frfr, np.angle(F, deg=True), ls='--')
ax[0].legend(loc='best',
            edgecolor='black',
            fancybox=True,
            shadow=True,
            framealpha=1,
            fontsize=8)
plt.show()

In [ ]:
print(com.nume)
print(com.deno)